In [1]:
from dataclasses import dataclass
import pandas as pd
from multicall import Call

from mainnet_launch.constants import (
    ALL_CHAINS,
    ROOT_PRICE_ORACLE,
    INCENTIVE_PRICING_STATS,
    LIQUIDATION_ROW,
    profile_function,
    ChainData,
    ETH_CHAIN,
)
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    identity_with_bool_success,
    safe_normalize_with_bool_success,
)
from mainnet_launch.database.schema.postgres_operations import _exec_sql_and_cache
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
    insert_avoid_conflicts,
)
from mainnet_launch.database.schema.views import get_token_details_dict

from mainnet_launch.database.schema.postgres_operations import get_subset_not_already_in_column, insert_avoid_conflicts
from mainnet_launch.database.schema.full import Tokens, IncentiveTokenSwapped
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_transactions import (
    ensure_all_transactions_are_saved_in_db,
)
from mainnet_launch.database.schema.ensure_tables_are_current.using_onchain.update_destinations_tokens_and_autopoolDestinations_table import (
    ensure_all_tokens_are_saved_in_db,
)

from mainnet_launch.database.schema.full import IncentiveTokenSwapped
from mainnet_launch.constants import *
from mainnet_launch.data_fetching.get_events import fetch_events
from mainnet_launch.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI
import pandas as pd

# What we care about for incentive tokens

- How accurate is our selling of incentive tokens acheived and oracle token prices
- How frequent is our selling of incentive tokens (for the base asset)
- How fequently are we claiming tokens,

    - be fine with false positives

In [ ]:
query = """

with swapped_events_with_blocks as (
get


)



"""

In [2]:
def _get_highest_swapped_event_already_fetched() -> dict:
    # stub
    highest_block_already_fetched = {}
    for liquidation_row in [LIQUIDATION_ROW, LIQUIDATION_ROW2]:
        for chain in ALL_CHAINS:
            highest_block_already_fetched[(chain, liquidation_row(chain))] = chain.block_autopool_first_deployed
    return highest_block_already_fetched


def _add_token_details(
    all_swapped_events: pd.DataFrame, token_to_decimals: dict, token_to_symbol: dict
) -> pd.DataFrame:
    all_swapped_events["sellTokenAddress"] = all_swapped_events["sellTokenAddress"].apply(
        lambda x: Web3.toChecksumAddress(x)
    )
    all_swapped_events["buyTokenAddress"] = all_swapped_events["buyTokenAddress"].apply(
        lambda x: Web3.toChecksumAddress(x)
    )
    all_swapped_events["sellTokenAddress_decimals"] = all_swapped_events["sellTokenAddress"].map(token_to_decimals)
    all_swapped_events["buyTokenAddress_decimals"] = all_swapped_events["buyTokenAddress"].map(token_to_decimals)
    all_swapped_events["sellTokenAddress_symbol"] = all_swapped_events["sellTokenAddress"].map(token_to_symbol)
    all_swapped_events["buyTokenAddress_symbol"] = all_swapped_events["buyTokenAddress"].map(token_to_symbol)

    all_swapped_events["sellAmount_normalized"] = all_swapped_events["sellAmount"] / (
        10 ** all_swapped_events["sellTokenAddress_decimals"]
    )
    all_swapped_events["buyAmount_normalized"] = all_swapped_events["buyAmount"] / (
        10 ** all_swapped_events["buyTokenAddress_decimals"]
    )
    all_swapped_events["buyTokenAmountReceived_normalized"] = all_swapped_events["buyTokenAmountReceived"] / (
        10 ** all_swapped_events["buyTokenAddress_decimals"]
    )

    return all_swapped_events


def ensure_incentive_token_swapped_events_are_saved_in_db() -> pd.DataFrame:
    highest_block_already_fetched = _get_highest_swapped_event_already_fetched()
    all_new_inentive_token_swapped_events = []

    for chain in ALL_CHAINS:
        all_swapped_events = []
        token_addresses_to_ensure_we_have_in_db = set()
        for liquidation_row in [LIQUIDATION_ROW, LIQUIDATION_ROW2]:
            start_block = highest_block_already_fetched[(chain, liquidation_row(chain))] + 1
            contract = chain.client.eth.contract(liquidation_row(chain), abi=DESTINATION_DEBT_REPORTING_SWAPPED_ABI)
            swapped_df = fetch_events(contract.events.Swapped, chain=chain, start_block=start_block)
            swapped_df["liquidation_row"] = liquidation_row(chain)
            swapped_df["chain_id"] = chain.chain_id

            if not swapped_df.empty:
                all_swapped_events.append(swapped_df)

            token_addresses_to_ensure_we_have_in_db.update(set(swapped_df["sellTokenAddress"].unique()))
            token_addresses_to_ensure_we_have_in_db.update(set(swapped_df["buyTokenAddress"].unique()))

        ensure_all_tokens_are_saved_in_db(list(token_addresses_to_ensure_we_have_in_db), chain)
        token_to_decimals, token_to_symbol = get_token_details_dict()

        if all_swapped_events:
            all_swapped_events = pd.concat(all_swapped_events)
            all_swapped_events = _add_token_details(all_swapped_events, token_to_decimals, token_to_symbol)
        else:
            all_swapped_events = pd.DataFrame()

        new_incentive_token_swapped_events = all_swapped_events.apply(
            lambda r: IncentiveTokenSwapped(
                tx_hash=r["hash"],
                log_index=int(r["log_index"]),
                chain_id=int(r["chain_id"]),
                sell_token_address=r["sellTokenAddress"],
                buy_token_address=r["buyTokenAddress"],
                sell_amount=float(r["sellAmount_normalized"]),
                buy_amount=float(r["buyAmount_normalized"]),
                buy_amount_received=float(r["buyTokenAmountReceived_normalized"]),
                liquidation_row=r["liquidation_row"],
            ),
            axis=1,
        ).tolist()

        all_new_inentive_token_swapped_events.extend(new_incentive_token_swapped_events)

        # ensure_all_transactions_are_saved_in_db(list(all_swapped_events["hash"].unique()), chain)
        # insert_avoid_conflicts(new_incentive_token_swapped_events, IncentiveTokenSwapped)

    return all_new_inentive_token_swapped_events


all_swapped_events = ensure_incentive_token_swapped_events_are_saved_in_db()
all_swapped_events

[IncentiveTokenSwapped(tx_hash='0xeaff1c935812795d7c751554c06a6f725d0523fe985d45b0c121f9600c2e602f', log_index=212, chain_id=1, liquidation_row='0xBf58810BB1946429830C1f12205331608c470ff5', sell_token_address='0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF', buy_token_address='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', sell_amount=405.8855646009787, buy_amount=0.06566090443774106, buy_amount_received=0.06566090443774109),
 IncentiveTokenSwapped(tx_hash='0xeaff1c935812795d7c751554c06a6f725d0523fe985d45b0c121f9600c2e602f', log_index=239, chain_id=1, liquidation_row='0xBf58810BB1946429830C1f12205331608c470ff5', sell_token_address='0xba100000625a3754423978a60c9317c58a424e3D', buy_token_address='0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', sell_amount=97.65686217963037, buy_amount=0.07358361751050911, buy_amount_received=0.07358361751050911),
 IncentiveTokenSwapped(tx_hash='0xeaff1c935812795d7c751554c06a6f725d0523fe985d45b0c121f9600c2e602f', log_index=263, chain_id=1, liquidation_row='0xBf588

In [7]:
mainnet_tokens = set(
    [t.sell_token_address for t in all_swapped_events if t.chain_id == 1]
    + [t.buy_token_address for t in all_swapped_events if t.chain_id == 1]
)


def _get_fast_and_slow_prices(success, data):
    if success:
        fast, slow = data
        return (fast, slow)
    return None


incentive_pricing_stats_calls = [
    Call(
        INCENTIVE_PRICING_STATS(ETH_CHAIN),
        ["getPrice(address,uint40)((uint256,uint256))", addr, 5 * 86400],  # 2 days of latency
        [(addr, _get_fast_and_slow_prices)],
    )
    for addr in mainnet_tokens
]


prices = get_state_by_one_block(incentive_pricing_stats_calls, ETH_CHAIN.client.eth.block_number - 100, ETH_CHAIN)
prices

{'0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B': (798700752901406,
  874659573569697),
 '0x6f40d4A6237C257fff2dB00FA0510DeEECd303eb': None,
 '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48': (220090677780083,
  221720205708104),
 '0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f': (219320282354428,
  221555644637884),
 '0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0': (1209496150228640007,
  1208805220873923322),
 '0xba100000625a3754423978a60c9317c58a424e3D': (292936124806019,
  299462653201314),
 '0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38': (1051656220704449371,
  1051502612082612912),
 '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2': None,
 '0x6B175474E89094C44Da98b954EedeAC495271d0F': (218676793208902,
  222052752404810),
 '0x865377367054516e17014CcdED1e7d814EDC9ce4': None,
 '0x58D97B57BB95320F9a05dC918Aef65434969c2B2': None,
 '0xD533a949740bb3306d119CC777fa900bA034cd52': (178639120548042,
  189082063199548),
 '0x6DF0E641FC9847c0c6Fde39bE6253045440c14d3': (2610218744959, 2576915432574),
 '0xC0c293ce4

In [ ]:
import aiohttp
import asyncio


# Get price from the Tokemak Labs API
async def get_price_from_tokemak_api(chain: ChainData, sell_token_address: str, quoteTokenAddress: str, timestamp: int):
    try:
        params = {
            "chainId": chain.chain_id,
            "systemName": "gen3",
            "token": sell_token_address,
            "timestamp": timestamp,
            "timeoutMS": 5000,
        }

        # Add denomination if quoteTokenAddress is provided
        if quoteTokenAddress and quoteTokenAddress != "0x0000000000000000000000000000000000000000":
            params["denominateIn"] = quoteTokenAddress
            params["denominateInDecimals"] = 18  # Default for most ERC20 tokens

        async with aiohttp.ClientSession() as session:
            async with session.get(
                "https://generic-swaps-prices-infra-staging.tokemak.workers.dev/price",
                params=params,
                timeout=aiohttp.ClientTimeout(total=6),  # Slightly longer than API timeout
            ) as response:
                if response.status == 200:
                    data = await response.json()
                    # Extract price from response - adjust based on actual API response structure
                    if "price" in data:
                        return data["price"]
                    else:
                        return None
                else:
                    return None
    except asyncio.TimeoutError:
        print(f"API timeout for token {sell_token_address}")
        return None
    except Exception as e:
        print(f"API error for token {sell_token_address}: {str(e)}")
        return None


prices = {}
current_unix = 1756409373

for t in mainnet_tokens:
    price = await get_price_from_tokemak_api(ETH_CHAIN, t, WETH(ETH_CHAIN), current_unix - 10 * 60)
    prices.append((ice))

In [10]:
prices

[('0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B', 0.000780128598127498),
 ('0x6f40d4A6237C257fff2dB00FA0510DeEECd303eb', 0.001419578105858051),
 ('0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48', 0.000223509778322259),
 ('0x40D16FC0246aD3160Ccc09B8D0D3A2cD28aE6C2f', 0.000223513201254919),
 ('0x7f39C581F595B53c5cb19bD0b3f8dA6c935E2Ca0', 1.2109484307583775),
 ('0xba100000625a3754423978a60c9317c58a424e3D', 0.000291499423226533),
 ('0xf1C9acDc66974dFB6dEcB12aA385b9cD01190E38', 1.0528467756284148),
 ('0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2', 0.9994265799032452),
 ('0x6B175474E89094C44Da98b954EedeAC495271d0F', 0.000223652788938433),
 ('0x865377367054516e17014CcdED1e7d814EDC9ce4', 0.000222729546897495),
 ('0x58D97B57BB95320F9a05dC918Aef65434969c2B2', 0.000476387987463001),
 ('0xD533a949740bb3306d119CC777fa900bA034cd52', 0.000178683150670416),
 ('0x6DF0E641FC9847c0c6Fde39bE6253045440c14d3', 2.552405216663e-06),
 ('0xC0c293ce456fF0ED870ADd98a0828Dd4d2903DBF', 4.3192494509811e-05),
 ('0x48C3399719

NameError: name 'token_addresses' is not defined

In [ ]:
if all_swapped_events.isna().any().any():
    raise ValueError("Some Swapped Events are missing")

In [ ]:
@dataclass
class Swapped:
    tx_hash: str  # primary keys
    log_index: int  # primary keys
    sell_token_address: str
    buy_token_address: str
    sell_amount: float
    buy_amount: float
    buyTokenAddress_amount_received: float
    liquidation_row_contract_address: str


@dataclass
class VaultLiquidated:
    """Reward tokens are sold for base asset. This at a per destiantion vault level"""

    tx_hash: str  # primary keys
    log_index: int  # primary keys

    destination_vault_address: str
    from_token_address: str
    to_token_address: str

    liquidated_amount: float  # in terms of to_token_address
    liquidation_row_contract_address: str


# https://sonicscan.org/tx/0x063b1707c4ad86daa03568bd1758dcbac73c4038d127e0342a1b11d555964622
# this tells me when a vault was liqudiated, (what and how much)
# we can tell what we price we got and when.

In [ ]:
# oracle_price_calls = [
#     Call(
#         INCENTIVE_PRICING_STATS(chain),
#         ["getPrice(address,uint40)((uint256,uint256))", addr, 2 * 86400],  # 2 days of latency
#         [(addr, _min_of_low_and_high_price)],
#     )
#     for addr in token_addresses
# ]


# emit Swapped(address(sellToken), address(buyToken), sellAmount, buyTokenAmountReceived, buyTokenAmountReceived);

# interface IAsyncSwapper {
#     error TokenAddressZero();
#     error SwapFailed();
#     error InsufficientBuyAmountReceived(uint256 buyTokenAmountReceived, uint256 buyAmount);
#     error InsufficientSellAmount();
#     error InsufficientBuyAmount();
#     error InsufficientBalance(uint256 balanceNeeded, uint256 balanceAvailable);

#     event Swapped(
#         address indexed sellTokenAddress,
#         address indexed buyTokenAddress,
#         uint256 sellAmount,
#         uint256 buyAmount,
#         uint256 buyTokenAmountReceived
#     );

#     /**
#      * @notice Swaps sellToken for buyToken
#      * @param swapParams Encoded swap data
#      * @return buyTokenAmountReceived The amount of buyToken received from the swap
#      */
#     function swap(
#         SwapParams memory swapParams
#     ) external payable returns (uint256 buyTokenAmountReceived);
# }


# swapped event

# event Swapped(
#     address indexed sellTokenAddress,
#     address indexed buyTokenAddress,
#     uint256 sellAmount,
#     uint256 buyAmount,
#     uint256 buyTokenAmountReceived
# );


# https://etherscan.io/tx/0x052b4231be3c2b28480b335085cad1c20ba838cccd5fc98e9c1d39e8502c9f11#eventlog

# VaultLiquidated (index_topic_1 address vault, index_topic_2 address fromToken, index_topic_3 address toToken, uint256 amount)View Source

# 133

# Topics
# 0 0x0272b5a6ff5cab190795f808ef35307240b8bc0011849cb8e15c093b40b22dfb
# 1: vault
# 0x0091Fec1B75013D1b83f4Bb82f0BEC4E256758CB # Tokemak-Dola USD Stablecoin-DOLA/sUSDe string
# 2: fromToken
# 0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B # CVX
# 3: toToken
# 0x865377367054516e17014CcdED1e7d814EDC9ce4 # dola
# Data


# amount :
# 583413391926390257

# at a vault level?


# https://etherscan.io/address/0xe2c7011866db4cc754f1b9b60b2f2999b5b54be4#code


# we also want reward added events